# Analysis of the historic records of stop and search data from MPS 


In [1]:
# Located at: link https://data.police.uk/data/

# Manually downloaded each file

# Consists of ~ 5000 excel files from the MPS database 
# From time range of 2021 to 2024

# data organised in folders of month - year
#each folder contains ~ 127 Excel files
# each folder has 3 files for each force
# 1.name-of-force-outcomes
# 2. name-of-force-stop-and-search
# 3. name-of-force-outcomes - street level crime

In [2]:
#Import all the neccessry packages
import os
import shutil
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import seaborn as sns

import sys #used for 
import csv
import glob # used to access specific path names

print ("it worked")

%time
%timeit

it worked
CPU times: user 1 μs, sys: 0 ns, total: 1 μs
Wall time: 2.15 μs




# Step 4: Import Excel Files into Pandas ########

In [3]:
import glob
import pandas as pd

Street_excels = glob.glob(r"/Users/nathanstuttard/Documents/Data Science Projects/Police Dataset/Police Dataset/Street/*.csv")

all_excels = glob.glob(r"/Users/nathanstuttard/Documents/DataScienceProjects/PoliceDataset/PoliceDataset/StopandSearch/*.csv")

# Define the folder path containing the CSV files
folder_path = r'/Users/nathanstuttard/Documents/Data Science Projects/Police Dataset/Police Dataset/Stop and Search'

# Identify the CSV files
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Initialize empty lists to collect DataFrames for each category
stop_and_search_dfs = []
outcomes_dfs = []
street_data_dfs = []

# Loop through files and append to the respective list
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    try:
        df = pd.read_csv(file_path)
    except Exception as e:
        print(f"Error reading {file}: {e}")
        continue

    # Append to the appropriate list based on the file name
    if 'search' in file.lower():
        stop_and_search_dfs.append(df)
    elif 'outcomes' in file.lower():
        outcomes_dfs.append(df)
    elif 'street' in file.lower():
        street_data_dfs.append(df)

# Concatenate all DataFrames at once after the loop
stop_and_search_df = pd.concat(stop_and_search_dfs, ignore_index=True) if stop_and_search_dfs else pd.DataFrame()
outcomes_df = pd.concat(outcomes_dfs, ignore_index=True) if outcomes_dfs else pd.DataFrame()
street_data_df = pd.concat(street_data_dfs, ignore_index=True) if street_data_dfs else pd.DataFrame()

# Inspect the loaded DataFrames
if not stop_and_search_df.empty:
    print(f"Stop and Search Data contains {stop_and_search_df.shape[0]} records.")
else:
    print("Stop and Search DataFrame was not loaded.")

if not outcomes_df.empty:
    print(f"Outcomes Data contains {outcomes_df.shape[0]} records.")
else:
    print("Outcomes DataFrame was not loaded.")

if not street_data_df.empty:
    print(f"Street Data contains {street_data_df.shape[0]} records.")
else:
    print("Street DataFrame was not loaded.")




Stop and Search Data contains 1178965 records.
Outcomes Data contains 10765151 records.
Street Data contains 13778488 records.


Inspect the street data

In [7]:
street_data_df.head(2)

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,c0a56f252ea9a873bdfddfcc11be403c67b61b4fb0254e...,2022-02,Cambridgeshire Constabulary,Cambridgeshire Constabulary,-0.836855,51.821263,On or near Supermarket,E01017708,Aylesbury Vale 012D,Shoplifting,Status update unavailable,NaN
1,badba1c78227f0fc28cf8eb990c6902b3b4cac2a4b4483...,2022-02,Cambridgeshire Constabulary,Cambridgeshire Constabulary,0.735361,52.408126,On or near Sloane Way,E01026476,Breckland 017C,Violence and sexual offences,Unable to prosecute suspect,NaN


In [5]:
print(f"Street Data contains {street_data_df.shape[0]} records.")

Street Data contains 13778488 records.


So, Thirteen million, seven hundred seventy-eight thousand, four hundred eighty-eight rows

In [6]:
street_data_df_cleaned = df.dropna("Lastoutcomecategory")
print(f"Street Data contains {street_data_df_cleaned.shape[0]} records.")

TypeError: DataFrame.dropna() takes 1 positional argument but 2 were given

1. Convert Categorical Variables into Numerical Representations

Machine learning models typically work with numbers, not categories. To include categorical variables like Reported by, Falls within, and LSOA name in your model, you need to convert them into numerical formats.
	•	Why? Algorithms like logistic regression or tree-based models need numerical inputs to calculate distances or splits.
	•	How?
	•	One-hot encoding: Create a new column for each category. For example, if Falls within has three categories (e.g., “Cambridgeshire Constabulary”, “Metropolitan Police”, “Thames Valley Police”), you create three columns. For a given row, one of these columns will have a value of 1, and the others will be 0.
Example: